In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Step 1: import libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,classification_report
from sklearn.preprocessing import MinMaxScaler
print('import ok')

In [ ]:
#Step 2: read in the data and do a preliminary analysis

df = pd.read_csv("../input/pima-indians-diabetes-database/diabetes.csv", 
                 names=['num_pregnant','glucose_conc','diastolic_bp','triceps_thick',
                        'serum_insulin','bmi','pedigree','age'
                        ,'class'],
                 header=0)

# df = pd.read_csv("../input/pima-indians-diabetes-database/diabetes.csv")
print(df.shape)
print(df.describe())

In [ ]:
df.describe()

In [ ]:
df

In [ ]:
df.head()

In [ ]:
#Step 3: Split into input_df and target dataframe. axis=0, row, axis=1, column.
input_df = df.drop('class', axis=1)
target = df['class']

print(input_df.shape, target.shape)

In [ ]:
#distribution of class - imbalance class with '1' having lesser count
target.value_counts()

In [ ]:
# We can see that there are 268 "yes"es - 1
# and there are 500 "No"s - 0

In [ ]:
# Parameters
# *arrayssequence of indexables with same length / shape[0]
# Allowed inputs are lists, numpy arrays, scipy-sparse matrices or pandas dataframes.

# test_sizefloat or int, default=None
# If float, should be between 0.0 and 1.0 and represent the proportion of the dataset to include in the test split. If int, represents the absolute number of test samples. If None, the value is set to the complement of the train size. If train_size is also None, it will be set to 0.25.

# train_sizefloat or int, default=None
# If float, should be between 0.0 and 1.0 and represent the proportion of the dataset to include in the train split. If int, represents the absolute number of train samples. If None, the value is automatically set to the complement of the test size.

# random_stateint, RandomState instance or None, default=None
# Controls the shuffling applied to the data before applying the split. Pass an int for reproducible output across multiple function calls. See Glossary.

# shufflebool, default=True
# Whether or not to shuffle the data before splitting. If shuffle=False then stratify must be None.

# stratifyarray-like, default=None
# If not None, data is split in a stratified fashion, using this as the class labels. Read more in the User Guide.

In [ ]:
#Step 4: Split feature and label sets to train and data sets - 70-30, random_state is desirable for reproducibility, stratify - same proportion as input data

# random_state = 10
X_train, X_test, y_train, y_test = train_test_split(input_df, target, test_size = 0.3, random_state = 10, stratify = target)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
#Question 2 - Normalize using MinMaxScaler to constrain values to between 0 and 1.

scaler = MinMaxScaler(feature_range = (0,1))

scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
#Step 5: Create a logistic regression classifier, default c=1

logreg = LogisticRegression(solver='liblinear')
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
print('Testing accuracy %s' % accuracy_score(y_test, y_pred))


In [ ]:
#look at the value under the '1' class (or 'yes') for the corresponding precision, recall and f1 score
print(classification_report(y_test, y_pred))

In [ ]:
#Question 4 -Rerunning above with resampled data - using oversampling

sm = SMOTE(random_state = 7)
X_train_sm, y_train_sm = sm.fit_resample(X_train, y_train)

print(pd.value_counts(pd.Series(y_train_sm)))

clf = logreg.fit(X_train_sm, y_train_sm)
print('Model accuracy is',clf.score(X_test, y_test))

In [ ]:
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
#Question 1,3 - get the descending sorted indices based on coefficient values
sorted_index = np.argsort(-logreg.coef_)

#get the feature_names
feature_names = input_df.columns

#get the names of the important features (largest to smallest)
print(feature_names.to_numpy()[sorted_index])